Декоратор - функция, которая принимает другую функцию и что то возвращает.

Пример в рамках некого декоратора @trace, который выводит на экран сообщение с информацией о вызове декорируемой функции.

In [1]:
def trace(func):
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    return inner

Применем его к дождественной функции

In [2]:
@trace
def identity(x):
    "I do nothing useful."
    return x

In [3]:
identity(42)

identity (42,) {}


42

Теперь есть проблема в том, что декорируемая функция не имеет ряда необходимых атрибутов (которые имела функция-оригинал), например документации:

In [4]:
help(identity)

Help on function inner in module __main__:

inner(*args, **kwargs)



Выхлд их этой ситуации достаточно муторный, необходимо передать правильные значения в атрибуты декорируемой функции:

In [5]:
def trace(func):
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    inner.__module__ = func.__module__
    inner.__name__ = func.__name__
    inner.__doc__ = func.__doc__
    return inner

Проверка

In [6]:
@trace
def identity(x):
    "I do nothing useful."
    return x

In [7]:
help(identity)

Help on function identity in module __main__:

identity(*args, **kwargs)
    I do nothing useful.



В стандартной библиотеки Python есть функция, которая делает все за нас.

In [3]:
import functools

In [9]:
def trace(func):
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    functools.update_wrapper(inner, func)
    return inner

Такимже функционалом обладает декоратор wraps: 

In [10]:
def trace(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    return inner

Следующий момент - отключение данной "трассировки".

Зведем переменную trace_enabled, с помощью которой будем все включать и выключать.

In [11]:
trace_enabled = False
def trace(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    return inner if trace_enabled else func

**Что происходит?** - в случае если trace выключен, то результатом применения декоратора является сама функция func - никаких дополнительных действий в момент ее исполнения происходить не будет. 

Далее у нас есть желание писать не в стандартный вывод, а в вывод для ошибок. Для этого необходимо сделать декоратор с аргументом. 

In [14]:
def trace(handle):
    def decorator(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            print(func.__name__, args, kwargs, file=handle)
            return func(*args, **kwargs)
        return inner
    return decorator

Тройная вложенность расстраивает и является трудно читаемой, в силу этого есть еще один подход (реализуемый также через декораторы), который позволяет упростить создание декораторов с аргументами.

In [3]:
def with_arguments(deco):
    @functools.wraps(deco)
    def wrapper(*dargs, **dkwargs):
        def decorator(func):
            result = deco(func, *dargs, **dkwargs)
            functools.update_wrapper(result, func)
            return result
        return decorator
    return wrapper

**Что происходит?** 

1. with_arguments принимает декоратор deco,
2. завторачивает его во wrapper, так как deco - декоратор с аргументами, а затем в decorator.
3. decorator конструирует новый декоратор с помощью deco и копирует в него внутренние атрибуты функции func.

И тогда:

In [7]:
@with_arguments
def trace(func, handle):
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs, file=handle)
        return func(*args, **kwargs)
    return inner

In [8]:
import sys
@trace(sys.stderr)
def identity(x):
    return x

И теперь, когда пришло осознание, что все это досаточно сложно, можно рассмотреть более упрощенное создание декораторов с аргументами.

In [10]:
# Декораторы с аргументами: магическая версия

def trace(func=None, *, handle=sys.stdout): # звездочка позволяет сделать handle ключевым аргументом. Без нее аргумент попаден в None
    # со скобками
    if func is None:
        return lambda func: trace(func, handle=handle)
    
    # без скобок
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    return inner

Декоратор - способ модифицироовать поведение функции, созраняя читаемость кода.

Они бывают:

+ без аргументов;
+ с аргументами
+ с опциональными аргументами

Пример использования декораторов: функция, вызывающая другую функцию определенное количество раз и выводящая минимальное время исполнения:

In [27]:
import time
def timethis(func=None, *, n_iter=100):
    if func is None:
        return lambda func: timethis(func, n_iter=n_iter)
    
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, end=" ... ")
        acc = float('inf')
        for i in range(n_iter):
            tick = time.perf_counter()
            result = func(*args, **kwargs)
            acc = min(acc, time.perf_counter() - tick)
        print(acc)
        return result
    return inner

In [28]:
result = timethis(sum)(range(10**6))

sum ... 0.04529533618096471


Еще один пример полезного декоратора -  функция, позволяющая сделать что-либо один раз:

In [31]:
def once(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        if not inner.called:
            func(*args, **kwargs)
            inner.called = True
    inner.called = False
    return inner    

In [32]:
@once
def initialize_settings():
    print("Settings initialized.")

In [33]:
initialize_settings()

Settings initialized.


In [34]:
initialize_settings() # Уже не сработает.

Следующий полезный декоратор - Меморизация - сохранение результатов выполнения функции для предотвращения избыточных вычислений. 

In [1]:
def memoized(func):
    cache = {}
    
    @functools.wraps(func)
    def inner(*args, **kwargs):
        key = args, kwargs
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]
    return inner
# Только она не будет работать. Далее пример функции акермана, демонстрирующий это 

In [4]:
@memoized
def akermann(m, n):
    if not m:
        return n + 1
    elif not n:
        return akermann(m - 1, 1)
    else:
        return akermann(m - 1, akermann(m , n -1))

In [5]:
akermann(3, 4)

TypeError: unhashable type: 'dict'

Частное решение проблемы

In [6]:
def memoized(func):
    cache = {}
    
    @functools.wraps(func)
    def inner(*args, **kwargs):
        key = args + tuple(sorted(kwargs.items()))
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]
    return inner

In [7]:
@memoized
def akermann(m, n):
    if not m:
        return n + 1
    elif not n:
        return akermann(m - 1, 1)
    else:
        return akermann(m - 1, akermann(m , n -1))

In [8]:
akermann(3, 4)

125

Данный подход реализуем только для тех случаем когда аргументы хэшируемы. В том случае, если аргументом будет являться словарь - все сломается. 

В Pyhton есть возможность добавления контрактного программирования, т.е. способа проектрирования программ, основывающегося на формальном описании интерфейсов в терминах предусловий, постусловий и инвариантов. подробнее об этом по ссылке: https://pypi.python.org/pypi/contracts

+ предусловие - то, что выполняется над аргументами перед вызовом функции;
+ постусловие - то, что выполняется над результатом вызова.

In [10]:
@pre(lambda x: x >= 0, 'negative arguments') # Если аргумент будет отрицательным, будет ошибка
def check_log(x):
    pass

NameError: name 'pre' is not defined

In [11]:
import math
is_not_nan = post(lambda r: not math.isnan(r), 'not a number') # проверит, а не вернула ли функция nan

NameError: name 'post' is not defined

Определим данные функции

In [12]:
def pre(cond, message):
    def wrapper(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            assert cond(*args, **kwargs), message
            return func(*args, **kwargs)
        return inner
    return wrapper

In [13]:
@pre(lambda x: x >= 0, 'negative arguments') # Если аргумент будет отрицательным, будет ошибка
def check_log(x):
    return math.log(x)

In [18]:
def post(cond, message):
    def wrapper(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            result = func(*args, **kwargs)
            assert cond(result), message
            return result
        return inner
    return wrapper

In [19]:
@post(lambda r: not math.isnan(r), 'not a number') # проверит, а не вернула ли функция nan
def something_useful():
    return float('nan')

In [20]:
something_useful()

AssertionError: not a number

Python позволяет применить к функции более чем один декоратор. Тем не менее, важно отметить, что порядок декораторов имеет значение.

In [22]:
def square(func):
    return lambda x: func(x * x)


def addsome(func):
    return lambda x: func(x + 42)

In [23]:
@square
@addsome
def identity(x):
    return x

In [24]:
identity(2)

46

In [25]:
@addsome
@square
def identity(x):
    return x

In [26]:
identity(2)

1936

Распишем, что происходит в случае первого примера:

    identity = square(addsome(identity))

Посмотрим, что происходит, когда мы вызываем функцию от аргумента:

    identity(2)
Сначала двойку получает **square**, т.е. на выходе 4. Затем прибавляется 42. в итоге 46.



#### Модуль functools

Первый полезный представитель модуля - lru_cache. Является декоратором и сохраняет данные последних вызовов.

In [27]:
@functools.lru_cache(maxsize=5)
def akermann(m, n):
    if not m:
        return n + 1
    elif not n:
        return akermann(m - 1, 1)
    else:
        return akermann(m - 1, akermann(m , n -1))

In [28]:
akermann(3, 4)

125

In [29]:
akermann.cache_info()

CacheInfo(hits=65, misses=315, maxsize=5, currsize=5)

Еще один полезный представитель модуля - функция partial, позволяющая зафиксировать часть позиционных и ключевых аргументов.

In [30]:
f = functools.partial(sorted, key=lambda p : p[1])

In [31]:
f([('a', 4), ('b', 2)])

[('b', 2), ('a', 4)]

Мегамощный singledispatch позволяет менять поведение функции в зависимости от типа переданного ей аргумента.

In [32]:
@functools.singledispatch
def pack(obj):
    type_name = type(obj).__name__
    assert False, "Unsupported type: " + type_name

Научим функцию pack сериализовывать списки и числа:

In [34]:
@pack.register(int)
def _(obj):
    return b"I" + hex(obj).encode("ascii")
@pack.register(list)
def _(obj):    
    return b"L" + b".".join(map(pack, obj))

In [35]:
pack([1, 2, 3])

b'LI0x1.I0x2.I0x3'

In [36]:
pack(5)

b'I0x5'

In [37]:
pack(42.0)

AssertionError: Unsupported type: float